In [1]:
import os, sys

import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options

import pandas as pd

In [2]:
# os.mkdir('data')

In [3]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True) # this keeps the window open post execution/errors/ changing frames (explained later)

prefs = {"download.default_directory": '/Users/justinpaul/Downloads',  # note need to change this
         "directory_upgrade": True}
chrome_options.add_experimental_option("prefs", prefs)

In [4]:
file = 'https://ai.engin.umich.edu/people/'


In [5]:
# driver = webdriver.Chrome('/usr/local/bin/chromedriver')
driver = webdriver.Chrome('/usr/local/bin/chromedriver', options=chrome_options)
driver.get(file)

/var/folders/md/xfvst6gj5bg055t0jwj54sy00000gn/T/ipykernel_35570/3316057419.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/local/bin/chromedriver', options=chrome_options)


In [6]:
%%time
li = []
for faculty in driver.find_elements(by=By.XPATH, value='//div[@class="eecs_person_copy"]'):
    
    # standard info
    relevant_info = {
        'name': faculty.find_element(By.XPATH, './/h4').text,
        'title': faculty.find_element(By.XPATH, './/span[@class="person_title_section"]').text,
        'email': faculty.find_element(By.XPATH, './/a[@class="person_email"]').text,
    }
    
    # optional info - interests
    try: val = faculty.find_element(By.XPATH, './/span[@class="person_copy_section pcs_tall"]').text \
        [20:].strip('.').split(', ')
    except: val = None
    relevant_info['interests'] = val
    
    # optional info - website
    try: val = faculty.find_element(By.XPATH, './/a[@class="person_web"]').get_attribute('href')
    except Exception as e: val = None
    relevant_info['website'] = val
    
    # optional info - phone, office
    stubs = ['phone', 'office']
    for opt_field in faculty.find_elements(By.XPATH, './/span[@class="person_copy_section "]'):
        text = opt_field.text
        for x in stubs:
            if text.startswith(f'{x.title()}: '):
                relevant_info[x] = text[len(x) + 2:]
    
    for x in stubs:
        if x not in relevant_info:
            relevant_info[x] = None
    # print(*relevant_info.items(), sep='\n')
    li.append(relevant_info)
    

CPU times: user 350 ms, sys: 31.8 ms, total: 381 ms
Wall time: 4.76 s


In [7]:
driver.close()

In [8]:
df = pd.DataFrame(li)

In [9]:
df.head()

,name,title,email,interests,website,phone,office
0,"Banovic, Nikola","Assistant Professor, Electrical Engineering an...",nbanovic@umich.edu,"[Computational Interaction, Human-centered Exp...",http://www.nikolabanovic.net/,734-764-4289,2630 Beyster Bldg.
1,"Baveja, Satinder Singh",Toyota Professor of Artificial Intelligence,baveja@umich.edu,"[Reinforcement Learning, Machine Learning, Com...",http://web.eecs.umich.edu/~baveja,(734) 936-2831,3765 Beyster Bldg.
2,"Burdick, Laura","Lecturer III, Electrical Engineering & Compute...",lburdick@umich.edu,"[Natural language processing, data science, ma...",None,(734) 936-1312,3824 Beyster
3,"Chai, Joyce","Professor, Electrical Engineering and Computer...",chaijy@umich.edu,"[Natural language processing, language groundi...",https://web.eecs.umich.edu/~chaijy/,734-764-3308,3632 Beyster
4,"Chakraborty, Mithun","Assistant Research Scientist, Electrical Engin...",dcsmc@umich.edu,"[Multi-Agent Systems, Computational Economics,...",https://sites.google.com/umich.edu/mithunchakra/,None,2709 Beyster Bldg.


In [10]:
li[0]

{'name': 'Banovic, Nikola',
 'title': 'Assistant Professor, Electrical Engineering and Computer Science',
 'email': 'nbanovic@umich.edu',
 'interests': ['Computational Interaction',
  'Human-centered Explainable AI',
  'Behavior-aware Interfaces'],
 'website': 'http://www.nikolabanovic.net/',
 'phone': '734-764-4289',
 'office': '2630 Beyster Bldg.'}

In [11]:
df.to_csv('data/ai_lab.csv', index=False)

In [12]:
# notes:
# manually cleaned some "interest categories"
# Stella Yu, Wei Hu, Justin Johnson, Danai Koutra, Emily Provost